# Importing the required libraries

In [38]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd
import functools
from nltk.tokenize import sent_tokenize
from pywsd.utils import lemmatize_sentence
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score




# Taking a chunk of file and reading the chunk

In [2]:
chunk_iter = pd.read_csv("D:/QTInternship/aws_review_sofware_dataset/aws_review_sofware_dataset.csv",chunksize=10000)

#chunk_iter = pd.read_csv('your_file.csv', chunksize=1000)
df = next(chunk_iter)  # Read the first chunk

In [3]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [4]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")


# Importing Sentence tokenize

In [5]:
# from nltk.tokenize import sent_tokenize

In [6]:
for i in range(df.shape[0]):
    l1= sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1

# Importing lemmatize

In [7]:
# !pip install pywsd
# from pywsd.utils import lemmatize_sentence


Warming up PyWSD (takes ~10 secs)... took 4.305618524551392 secs.


In [8]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [9]:
df["words_sentences"] = "default"

In [10]:
# import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df.loc[k,"words"])

# Importing Count Vectorizer

In [11]:
# from sklearn.feature_extraction.text import  CountVectorizer

In [12]:
df1=df

no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df1.words_sentences)

# Splitting the data and Encoding the target variable

In [13]:
df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())

In [14]:
df_y=df["verified"]

In [15]:

from sklearn.preprocessing import LabelEncoder

In [16]:

le=LabelEncoder()

In [17]:
df_y_1=pd.DataFrame(df_y)

In [18]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [19]:
df_y_enc.columns

Index(['verified'], dtype='object')

In [20]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


# Splitting the data into Train and Test data

In [32]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score

X_Train,X_Test,Y_Train,Y_Test = train_test_split(df_x,df_y_enc,test_size=0.3,random_state=43,stratify=df_y_enc)



# Model Building

In [33]:
RFC = RandomForestClassifier(random_state=43,n_estimators=1000)
RFC.fit(X_Train, Y_Train)

Y_pred_Train=RFC.predict(X_Train)
accuracy_train = accuracy_score(Y_Train, Y_pred_Train)
print(f"Accuracy of Training data is: {accuracy_train:.2f}")

print(confusion_matrix(Y_Train,Y_pred_Train))

# Generate a classification report
report = classification_report(Y_Train, Y_pred_Train)
print(f"Classification Report:\n{report}")



Y_pred_Test=RFC.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\AppData\Local\Temp\ipykernel_11052\1043178231.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RFC.fit(X_Train, Y_Train)


Accuracy of Training data is: 1.00
[[4228    4]
 [   0 2768]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4232
           1       1.00      1.00      1.00      2768

    accuracy                           1.00      7000
   macro avg       1.00      1.00      1.00      7000
weighted avg       1.00      1.00      1.00      7000

Accuracy of Test data is: 0.77
[[1587  227]
 [ 469  717]]
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.87      0.82      1814
           1       0.76      0.60      0.67      1186

    accuracy                           0.77      3000
   macro avg       0.77      0.74      0.75      3000
weighted avg       0.77      0.77      0.76      3000



In [34]:
modelgbc = GradientBoostingClassifier(random_state=43,learning_rate=0.1,n_estimators=1000)
modelgbc.fit(X_Train, Y_Train)
Y_pred_Test=modelgbc.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Test data is: 0.77
[[1493  321]
 [ 356  830]]
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.82      0.82      1814
           1       0.72      0.70      0.71      1186

    accuracy                           0.77      3000
   macro avg       0.76      0.76      0.76      3000
weighted avg       0.77      0.77      0.77      3000



In [36]:
MNB = MultinomialNB()
MNB.fit(X_Train, Y_Train)
Y_pred_Test=MNB.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

Accuracy of Test data is: 0.69
[[1254  560]
 [ 366  820]]
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.69      0.73      1814
           1       0.59      0.69      0.64      1186

    accuracy                           0.69      3000
   macro avg       0.68      0.69      0.68      3000
weighted avg       0.70      0.69      0.69      3000



C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
